# Solving and Simulating a DSGE Model using the LinApp Tools in Python

This notebook explains how to set up a DSGE model and solve it it Python using the linearization techniques implemented in the LinApp package.

The LinApp package includes the following files:

LinApp_Deriv - Takes a function, funcname.py, as an input.   Generates the derivative matrices needed for the Uhlig toolkit.

LinApp_Solve - Takes the derivative matrices as inputs.  Uses the code from the Uhlig toolkit to generate the coefficients for the linearized policy function(s).

### We begin by writing down the behavioral equations for a simple DSGE model.

We use the following definitions:

$k_t$ is the capital stock in period $t$.

$\ell_t$ is the labor supplied in period $t$.

$z_t$ is the percent deviation of technology from its long-run value.

$r_t$ is the rental rate on capital.

$w_t$ is the wage rate.

$c_t$ is private consumption.

$i_t$ is investment.

$\varepsilon_t$ is the random innovation to $z_t$

We also have the following parameters:
$\alpha$ is the capital share in GDP.

$\beta$ is the subjective discount factor.

$\gamma$ is the curvature of the utility function.  In our example this is the constant elasticity of intertemporal substitition.  It is also the constant coefficient of relative risk aversion.

$\delta$ is the rate of capital depreciation.

$\rho$ is the autocorrelation of $z_t$.

$\sigma$ is the standard deviation of the $\varepsilon_t$ shocks.

Suppose the household's Bellman equation is given by:
$$ V(k_t, z_t) = \max_{k_{t+1}} \frac{c_t^{1-\gamma}-1}{1-\gamma} - \chi \frac{1}{1+\theta}\ell^{1+\theta} + \beta V(k_{t+1}, z_{t+1}) $$

$$ c_t = w_t + (1+r_t-\delta)k_t - k_{t+1} $$

The first-order conditions are:
$$ c_t^{-\gamma} = \beta V_k(k_{t+1}, z_{t+1}) $$
$$ c_t^{-\gamma} w_t = \chi \ell^\theta $$
The envelope condition is:
$$ \beta V_k(k_t, z_t) = c_t^{-\gamma}(1+r_t-\delta) $$
Combining give the Euler equation:
$$ c_t^{-\gamma} = \beta c_{t+1}^{-\gamma}(1+r_{t+1}-\delta) $$

From the firm's problem we have the following:
$$ y_t = k_t^\alpha e^{(1-\alpha)z_t} $$
$$ r_t = \alpha \frac{y_t}{k_t} $$
$$ w_t = (1-\alpha)y_t $$

Finally, we assume the following law of motion for $ z_t $:
$$ z_t = \rho z_{t-1} + \varepsilon_t; \varepsilon_t \sim iid(0,\sigma^2) $$

## Programming Definitions Functions

We want to write a function, $\Gamma(X_{t+1}, X_t, X_{t-1}, Y_{t+1}, Y_t, Z_{t+1}, Z_t) = 0 $ based on the Euler equation.  We will define our set of endogenous state variables as $X_t = k_{t+1}$, $Y_t = \ell_t$, and $Z_t = z_t$.

First we can define $y_t, r_t, w_t$ and $c_t$ as functions of the state $(k_t, z_t)$ using the budget constraint and the conditions from the firm's problem.  The parameters of the model, $\alpha, \beta, \gamma, \delta, \rho, \& \sigma$, are included in the Python list `mparams`.

$$ y_t = k_t^\alpha e^{(1-\alpha)z_t} $$
$$ r_t = \alpha \frac{y_t}{k_t} $$
$$ w_t = (1-\alpha)y_t $$
$$ c_t = w_t + (1+r_t-\delta)k_t - k_{t+1} $$
$$ i_t = y_t - c_t $$
$$ u_t = \frac{c_t^{1-\gamma}-1}{1-\gamma} - \chi \frac{1}{1+\theta}\ell^{1+\theta}$$

In [1]:
def Modeldefs(Xp, X, Y, Z, params):
    '''
    This function takes vectors of endogenous and exogenous state variables
    along with a vector of 'jump' variables and returns explicitly defined
    values for consumption, gdp, wages, real interest rates, and transfers
    
    Inputs are:
        Xp: value of capital in next period
        X: value of capital this period
        Y: value of labor this period
        Z: value of productivity this period
        params: list of parameter values
    
    Output are:
        Y: GDP
        w: wage rate
        r: rental rate on capital
        T: transfer payments
        c: consumption
        u: utiity
    '''
    
    # unpack input vectors
    kp = Xp
    k = X
    ell = Y
    z = Z
    
    # find definintion values
    Y = k**alpha*(np.exp(z)*ell)**(1-alpha)
    w = (1-alpha)*Y/ell
    r = alpha*Y/k
    c = (w*ell + (r - delta)*k) + k - kp
    u = c**(1-gamma)/(1-gamma) - chi*ell**(1+theta)/(1+theta)
    return Y, w, r, c, u

Next we define our $\Gamma$ function which is simply the Euler equation rewritten.
$$ \Gamma_1 = c_t^{-\gamma}w_t - \chi \ell^\theta $$
$$ \Gamma_2 = c_t^{-\gamma} - \beta c_{t+1}^{-\gamma} (1+r_{t+1}-\delta) $$


In [2]:
def Modeldyn(theta0, params):
    '''
    This function takes vectors of endogenous and exogenous state variables
    along with a vector of 'jump' variables and returns values from the
    characterizing Euler equations.
    
    Inputs are:
        theta: a vector containng (Xpp, Xp, X, Yp, Y, Zp, Z) where:
            Xpp: value of capital in two periods
            Xp: value of capital in next period
            X: value of capital this period
            Yp: value of labor in next period
            Y: value of labor this period
            Zp: value of productivity in next period
            Z: value of productivity this period
        params: list of parameter values
    
    Output are:
        Euler: a vector of Euler equations written so that they are zero at the
            steady state values of X, Y & Z.  This is a 2x1 numpy array. 
    '''
    
    # unpack theat0
    (Xpp, Xp, X, Yp, Y, Zp, Z) = theta0
    
    # find definitions for now and next period
    Y, w, r, c, u = Modeldefs(Xp, X, Y, Z, params)
    Yp, wp, rp, cp, up = Modeldefs(Xpp, Xp, Yp, Zp, params)
    
    # Evaluate Euler equations
    ell = Y
    E1 = (c**(-gamma)*w) / (chi*ell**theta) - 1
    E2 = (c**(-gamma)) / (beta*cp**(-gamma)*(1 + rp - delta)) - 1
    
    
    return np.array([E1, E2])

The advantage of writing the code with two functions is that we only need to code the definitions once.  We call that function twice, but we minimize the number of functions we need to code.


## Writing the Main Program

Now we can write a program that will solve and simulate our model.
First we need to call the Python packages and files we need.

In [3]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy.optimize as opt
from LinApp_FindSS import LinApp_FindSS
from LinApp_Deriv import LinApp_Deriv
from LinApp_Solve import LinApp_Solve

Next we set our parameter values.

In [4]:
# set parameter values
alpha = .35
beta = .99
gamma = 2.5
delta = .08
chi = 10.
theta = 2.
rho = .9
sigma = .02

# make parameter list to pass to functions
params = (alpha, beta, gamma, delta, chi, theta, rho, sigma)

# set LinApp parameters
Zbar = np.array([0.])
nx = 1
ny = 1
nz = 1
logX = 0
Sylv = 0

### Finding the Steady State

In [5]:
# take a guess for steady state values of k and ell
guessXY = np.array([.1, .25])

# find the steady state values using LinApp_FindSS
XYbar = LinApp_FindSS(Modeldyn, params, guessXY, Zbar, nx, ny)
(kbar, ellbar) = XYbar
print ('XYbar: ', XYbar)

# set up steady state input vector
theta0 = np.array([kbar, kbar, kbar, ellbar, ellbar, 0., 0.])

# check SS solution
check = Modeldyn(theta0, params)
print ('check: ', check)
if np.max(np.abs(check)) > 1.E-6:
    print ('Have NOT found steady state')

XYbar:  [ 3.06094463  0.37947314]
check:  [  5.48761037e-12   2.22044605e-16]


We can get the steady states for all the other variables using our definintions function

In [6]:
Ybar, wbar, rbar, cbar, ubar = Modeldefs(kbar, kbar, ellbar, 0, params)
print ('Ybar: ', Ybar)
print ('wbar: ', wbar)
print ('rbar: ', rbar)
print ('cbar: ', cbar)
print ('ubar: ', ubar)

Ybar:  0.787983436396
wbar:  1.34973779755
rbar:  0.090101010101
cbar:  0.543107866251
ubar:  -1.84778102019


### Solving for the Policy Function Paramters

The next step is to find the derivatives of the Gamma function and then use these to solve for the coefficients of the linear policy function.  This requires using LinApp_Deriv and LinApp_Solve.

In [7]:
# find the derivatives matrices
[AA, BB, CC, DD, FF, GG, HH, JJ, KK, LL, MM, WW, TT] = \
    LinApp_Deriv(Modeldyn, params, theta0, nx, ny, nz, logX)
print('FF: ', FF)
print('GG: ', GG)
print('HH: ', HH)
print('LL: ', LL)
print('MM: ', MM)

# set value for NN    
NN = rho
    
# find the policy and jump function coefficients
PP, QQ, UU, RR, SS, VV = \
    LinApp_Solve(AA,BB,CC,DD,FF,GG,HH,JJ,KK,LL,MM,WW,TT,NN,Zbar,Sylv)
print ('PP:',PP)
print ('QQ', QQ)
print ('RR:',RR)
print ('SS', SS)

FF:  [[-4.60313716]]
GG:  [[ 9.27171253]]
HH:  [[-4.6496335]]
LL:  [[ 2.29969729]]
MM:  [[-2.35767729]]
Sylvester equation solver condition is not satisfied; proceed with the original method...
PP: [[ 0.86936886]]
QQ [[ 0.20014313]]
RR: [[-0.07216565]]
SS [[-0.19755317]]
